<a name="top"></a><img src="source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

运行SpinalHDL代码之前，务必加载SpinalHDL的库 
（**注**：*第一次运行时需要下载库，可能会有点慢，请耐心等待*）

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

## 参数化滤波器
**功能需求**  
- 滤波器有多级（默认3级），每一级为对称n（默认33）阶滤波器
- 每一级支持上采样和下采样，采样倍数1、2、3、4、5可配
- 每一级支持bypass
- 每一级系数可配
- 每一级增益可配
- 滤波器的数据输出方式
  + 每级滤波器支持单独冲刷
  + 每级滤波器要补零将尾巴输出
  + 滤波器级联连续输出

### 系统框图
![image.png](./slides/img/1.png)

### FIR 滤波器  
![image.png](./slides/img/2.png)
**主要分为两块**  
- FIRCtrl 负责上采样和下采样的控制，以及前后滤波器的握手
- FIRCore 负责单个样点的滤波计算
  + 寄存器组（Shift寄存器组，参数寄存器，系数寄存器）
  + 乘累加运算模块（对于33阶滤波器，1次乘累加需要17拍+1拍的握手开销）

用参数来定义一个FIRConfig
```scala
case class FIRConfig(iqWidth: Int = 8,
                     coefWidth: Int = 8,
                     tapNumbers: Int = 33,
                     maxUpSmpTimes : Int = 5
                     )
```

```scala
{
  def cyclesPerSmp(): Int = (tapNumbers>>1) + 1
  def cyclesPerSmp_(): Int = cyclesPerSmp - 1
```

```scala
  val accWidth: Int = iqWidth + coefWidth + log2Up(tapNumbers)
  val fixSelWidth: Int = log2Up(coefWidth + log2Up(tapNumbers))
}
```

**有了FIR的，再来构建Top层滤波器组的参数定义**
```scala
case class FilterGroupConfig(stagesNumber: Int = 3,
                             hwFreq: HertzNumber = 200 MHz,
                             sampleFreq: HertzNumber = 1.92 MHz){
  val firsConfig: Array[FIRConfig] = Array.fill(stagesNumber)(FIRConfig(8,8,33))
}
```

改变FIR的参数配置
```scala
val u3_fgc = FilterGroupConfig(3, 200 MHz, 1.92 Mhz)
u3_fgc.firsConfig(0) = FIRConfig(8,8,33,6)
u3_fgc.firsConfig(1) = FIRConfig(16,16,21,5)
u3_fgc.firsConfig(2) = FIRConfig(8,8,63,3)
```

定义U4项目的滤波器组
```scala
val u4_fgc = FilterGroupConfig(5, 400 MHz, 1.92 Mhz)
for(i <- 0 until u4_fgc.length){
    u4_fgc.firsConfig(i) = FIRConfig(16,16,23,5)
}
//Array.copy(Array.fill(5)(FIRconfig(16,16,23,5)),0,u4_fgc.firsConfig,0,5)
```

**有了参数，创建FIR模块**
```scala
case class FIR(fc: FIRConfig) extends Component {
  val io = new Bundle {
    val flush = in Bool()
    val bmi = slave(Bmi(fc.bmiConfig))
    val ins = slave Stream (IQ(fc.iqWidth bits))
    val ots = master Stream (IQ(fc.iqWidth bits))
  }
```

```scala
  val u_ctrl = new FIRCtrl(fc)
```

```scala
  u_ctrl.io.flush := io.flush
  u_ctrl.io.ready := io.ots.ready
```

```scala
  val u_core = new FIRCore(fc)
  u_core.io.ins      << u_ctrl.io.iq2Core
  u_core.io.tapRaddr << u_ctrl.io.tapRaddr
  u_core.io.bmi      << io.bmi
  u_core.io.flush    := io.flush
  u_ctrl.io.swif     := u_core.io.swif
```

```scala
when(u_core.io.swif.bypass) {
    io.ots << io.ins
  }.otherwise{
    io.ots << u_core.io.ots.toStream.queue(1)
  }
}
```

至此，我们就可以参数化一个FIRconfig
```scala
SpinalVerilog(new FIR(FIRConfig()))
```

改变参数，生成不同规格的FIR滤波器 
```scala
SpinalVerilog(new FIR(FIRConfig(iqWidth = 16,
                                coefWidth = 10,
                                tapNumbers = 33,
                                maxUpSmpTimes = 10)))
```

改变全局配置，定义时钟信息，生成目录等等
```scala
val spc = SpinalConfig(
    mode = Verilog,
    defaultConfigForClockDomains = ClockDomainConfig(
        resetKind = ASYNC,
        clockEdge = RISING, 
        resetActiveLevel = LOW),
    targetDirectory="rtl/")
spc.generate(new FIR(FIRConfig()))
```